In [33]:
import joblib
import pandas as pd
from feast import FeatureStore
from datetime import datetime
from pathlib import Path

In [34]:
home_dir = str(Path.home())

repo_path = home_dir
model_filename = f"{home_dir}/model.bin"
encoder_filename = f"{home_dir}/encoder.bin"

In [35]:
# load model

if not Path(model_filename).exists() or not Path(encoder_filename).exists():
    print("You need to train the model first!")

classifier = joblib.load(model_filename)

In [36]:
fs = FeatureStore(repo_path=repo_path)   # create Feast object

In [37]:
# Make online prediction (using DynamoDB for retrieving online features)
request = {
    "zipcode": [76104],
    "dob_ssn": ["19630621_4278"],
    "person_age": [133],
    "person_income": [59000],
    "person_home_ownership": ["RENT"],
    "person_emp_length": [123.0],
    "loan_intent": ["PERSONAL"],
    "loan_amnt": [35000],
    "loan_int_rate": [16.02],
}

In [46]:
# Make online prediction (using DynamoDB for retrieving online features)
request = {
    "zipcode": 76104,
    "dob_ssn": "19630621_4278",
    "person_age": 133,
    "person_income": 59000,
    "person_home_ownership": "RENT",
    "person_emp_length": 123.0,
    "loan_intent": "PERSONAL",
    "loan_amnt": 35000,
    "loan_int_rate": 16.02,
}

In [49]:
# Get online features from Feast
feature_vector = fs.get_online_features(
    entity_rows=[
        {
            "zipcode": request["zipcode"],
            "dob_ssn": request["dob_ssn"],
            "person_home_ownership": request["person_home_ownership"],
            "loan_intent": request["loan_intent"],
        }
    ],
    features=[
        "zipcode_features:city",
        "zipcode_features:state",
        "zipcode_features:location_type",
        "zipcode_features:tax_returns_filed",
        "zipcode_features:population",
        "zipcode_features:total_wages",
        "credit_history:credit_card_due",
        "credit_history:mortgage_due",
        "credit_history:student_loan_due",
        "credit_history:vehicle_loan_due",
        "credit_history:hard_pulls",
        "credit_history:missed_payments_2y",
        "credit_history:missed_payments_1y",
        "credit_history:missed_payments_6m",
        "credit_history:bankruptcies",
        "transformed_conv_rate:person_home_ownership_enc",
        "loan_intent:loan_intent_enc",
    ],
).to_dict()

In [50]:
print(feature_vector)

{'zipcode': [76104], 'dob_ssn': ['19630621_4278'], 'location_type': ['PRIMARY'], 'total_wages': [142325465], 'state': ['TX'], 'population': [10534], 'city': ['FORT WORTH'], 'tax_returns_filed': [6058], 'vehicle_loan_due': [11506], 'missed_payments_1y': [0], 'bankruptcies': [0], 'mortgage_due': [378847], 'missed_payments_2y': [0], 'missed_payments_6m': [0], 'student_loan_due': [44375], 'hard_pulls': [1], 'credit_card_due': [3343], 'person_home_ownership_enc': [0]}


In [15]:
# Join features to request features

features = request.copy()
del features['person_home_ownership']
del features['loan_intent']
features.update(feature_vector)
features_df = pd.DataFrame.from_dict(features)

In [16]:
features_df.head()

,zipcode,dob_ssn,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,location_type,...,tax_returns_filed,vehicle_loan_due,missed_payments_1y,bankruptcies,mortgage_due,missed_payments_2y,missed_payments_6m,student_loan_due,hard_pulls,credit_card_due
0,76104,19630621_4278,133,59000,RENT,123.0,PERSONAL,35000,16.02,PRIMARY,...,6058,11506,0,0,378847,0,0,44375,1,3343


In [17]:
# Apply ordinal encoding to categorical features

categorical_features = [
    "city",
    "state",
    "location_type",
]

# load encoder
encoder = joblib.load(encoder_filename)

# apply encoding
features_df[categorical_features] = encoder.transform(
    features_df[categorical_features]
)

In [18]:
features_df.head()

,zipcode,dob_ssn,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,location_type,...,tax_returns_filed,vehicle_loan_due,missed_payments_1y,bankruptcies,mortgage_due,missed_payments_2y,missed_payments_6m,student_loan_due,hard_pulls,credit_card_due
0,76104,19630621_4278,133,59000,-1.0,123.0,-1.0,35000,16.02,-1.0,...,6058,11506,0,0,378847,0,0,44375,1,3343


In [19]:
# Sort columns
features_df = features_df.reindex(sorted(features_df.columns), axis=1)

# Drop unnecessary columns
features_df = features_df[features_df.columns.drop("zipcode").drop("dob_ssn")]

# Make prediction
features_df["prediction"] = classifier.predict(features_df)

result = features_df["prediction"].iloc[0]

In [20]:
if result == 0:
    print("Loan approved!")
elif result == 1:
    print("Loan rejected!")

Loan rejected!
